## Data Importation

In [168]:
import pandas as pd
import numpy as np

train = pd.read_csv('./Dataset/train.csv')
test = pd.read_csv('./Dataset/test.csv')

In [169]:
train.shape

(891, 12)

In [170]:
test.shape
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [171]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [172]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


let's see infos 

In [173]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


We may want to know how many people survived in each class. We can use the groupby method to quickly do this:

In [174]:
pclass = train.groupby('Pclass').Survived.value_counts() 
pclass

Pclass  Survived
1       1           136
        0            80
2       0            97
        1            87
3       0           372
        1           119
Name: count, dtype: int64

as we can see , the statistics show that the survival rate of the passengers in the first class is higher than the second and third class. we can exploit this info 

## Feature Engineering

Let's create some additional features from the existing ones ( feature engineering )

In [175]:
train['Family_Size']=train['SibSp']+train['Parch']
test['Family_Size']=test['SibSp']+test['Parch']


In [176]:
corr_matrix = train.corr(numeric_only=True)

In [177]:
corr_matrix["Survived"].sort_values(ascending=False)

Survived       1.000000
Fare           0.257307
Parch          0.081629
Family_Size    0.016639
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Name: Survived, dtype: float64

As we can see , it seems that the Fare and Pclass are the most correlated attributes with the Survived attribute.

let's prepare our train set now

In [178]:
train_X = train.drop("Survived", axis=1)
train_y = train["Survived"].copy()

## StratifiedSampling

In [179]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(train, train["Survived"]):
 strat_train_set = train.loc[train_index]
 strat_test_set = train.loc[test_index]

In [180]:
strat_test_set["Survived"].value_counts() / len(strat_test_set)

Survived
0    0.614525
1    0.385475
Name: count, dtype: float64

In [181]:
for set_ in (strat_train_set, strat_test_set):
 set_.drop("Survived", axis=1, inplace=True)

In [182]:
train = strat_train_set.copy()
test = strat_test_set.copy()

## Pipeline Creation

let's create a pipline that prepares the data for training. but before let devise our dataset into numerical and categorical sub datasets

In [183]:
num_features=['Age','SibSp','Parch','Fare','Family_Size']
cat_features=['Sex']

In [184]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="most_frequent")),
 ('std_scaler', StandardScaler()),
 ])

In [185]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

full_pieline = ColumnTransformer([("num_pipeline", num_pipeline, num_features),("cat", OneHotEncoder(), cat_features)])
train_prepared = full_pieline.fit_transform(train_X)
test_prepared = full_pieline.fit_transform(test) 

## Training Model

In [186]:
from random import randint
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
from sklearn.model_selection import GridSearchCV
rs_space={    'n_neighbors':[30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
               "leaf_size": [30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
              'weights':["uniform", "distance"],
              "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
              'metric': ['euclidean', 'manhattan']
         }
random_search = GridSearchCV(knn, param_grid=rs_space, scoring='accuracy', refit=True, cv=5, verbose=2, n_jobs=-1)
random_search.fit(train_prepared, train_y)



Fitting 5 folds for each of 3600 candidates, totalling 18000 fits


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [30, 35, 40, 45, 50, 55, 60, 65, 70, 75,
                                       80, 85, 90, 95, 100],
                         'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': [30, 35, 40, 45, 50, 55, 60, 65, 70, 75,
                                         80, 85, 90, 95, 100],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy', verbose=2)

In [187]:
random_search.best_params_

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'euclidean',
 'n_neighbors': 30,
 'weights': 'uniform'}

In [188]:
model = KNeighborsClassifier(**random_search.best_params_)

Let's train our model now

model.fit(train)

In [189]:
model.fit(train_prepared, train_y)
predictions = model.predict(test_prepared)
predictions

array([0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0], dtype=int64)

let's now save the submission 

In [190]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [191]:
output.head()

,PassengerId,Survived
565,566,0
160,161,0
553,554,0
860,861,0
241,242,1
